In [ ]:
import numpy as np
import pandas as pd
import tokenize
import suji as suji
from crawling import getText, getOneText

users = getText('http://114.71.103.80/new_db.php?q=select+*+from+users%0D%0Alimit+10%0D%0A%0D%0A', '1')
del users[0]
idx=[]
for i in range(70):
    idx.append(i)
df = pd.DataFrame(columns=list(users), index=idx)
for user in users:
    problems = getText('http://114.71.103.80/new_db.php?q=select+*+from+code_submission+c%2C+%28select+id%2C+max%28regtime%29+a+from+code_submission+where+id+%3D'+user+'+group+by+p_no%29+d+where+c.id+%3D+d.id+and+d.a+%3D+c.regtime+',  '4')
    results=[]
    for problem in problems:
        with open('hello.py', 'wb+') as fw:
            fw.write(problem.encode('utf-8'))

        with open('hello.py', 'rb') as fr:
            tokens = tokenize.tokenize(fr.readline)
            try:
                results.append(suji.EqOpBl(tokens))
            except:
                pass
    df[user]=results

In [2]:
df

,201411023,201411085,201411130,201411153,201511020,201511057,201511080,201511153,201611074
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
